In [1]:
%matplotlib inline

import glob
import json
import more_itertools
import nibabel
import nilearn
import numpy as np
import os
import pandas as pd
import re
import scipy
import seaborn as sns
import sys

from load_confounds import Minimal
from joblib import parallel_backend
from numpy import nan as NaN
from matplotlib import pyplot as plt
from nibabel.nifti1 import Nifti1Image
from os import PathLike
from pathlib import Path, PosixPath
from random import sample
from nilearn.glm.first_level import FirstLevelModel, check_events
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn import image as nimage
from nilearn.input_data import MultiNiftiMasker, NiftiLabelsMasker
from nilearn.input_data import NiftiMapsMasker, NiftiMasker, NiftiSpheresMasker
from tqdm import tqdm
from typing import Iterable, Sequence, Union
# from nilearn import plotting as niplot
# from nilearn import image as nimage

from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import make_scorer, recall_score
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.utils import Bunch

from cimaq_decoding_utils import get_difumo, get_difumo_cut_coords
from cimaq_decoding_utils import get_fmri_sessions, fetch_fmriprep_session
# from cimaq_decoding_utils import clean_scale_mask_img

#libraries need to be installed in conda environment with pip install

from sklearn.feature_extraction import image
from sklearn.feature_extraction.image import extract_patches_2d
# from sklearn.image.io import imread

/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [3]:
!export PYTHONPATH="${PYTHONPATH}:/home/fnadeau/.linuxbrew/lib/python3.10/site-packages"

In [2]:
import imageio
# sorted(dict(os.environ).keys())

ModuleNotFoundError: No module named 'imageio'

In [9]:
impaths = sorted(filter(os.path.isfile, sorted(Path('/data/simexp/fnadeau/WMStim/').iterdir())))
# import PIL.Image as pilimage

imlist = []
for apath in impaths:
    with pilimage.open(apath) as imfile:
        image = pilimage.load(imfile)
        imlist.append((os.path.basename(imfile, image)))
        pilimage.close(imfile)

imdict = dict(tuple(imlist))
# sorted(dir(pilimage))
# with pil
# help(pilimage.open)
# # help(extract_patches_2d)
# sorted(dir(image))
# patches = extract_patches_2d(one_image, (2, 2))

AttributeError: module 'PIL.Image' has no attribute 'load'

In [ ]:


fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/'
events_dir = '/data/simexp/CIMAQ_AS_BIDS/'
atlases_dir = '../../nilearn_atlases/'



subjects = fetch_cimaq(topdir=fmriprep_dir, events_dir=events_dir, n_ses=2).data
sub00, sub01 = tuple(subjects)


In [ ]:
def trial_fmri(fmri_path:Union[str,os.PathLike, Nifti1Image],
               events_path:Union[str,os.PathLike, pd.DataFrame],
               sep:str='\t', t_r:float=None,
               **kwargs):
    from itertools import starmap
    from more_itertools import flatten
    from nilearn import image as nimage
    import pandas as pd
    # Make pandas Intervals (b:list of beginnigs, e:list of ends)
    mkintrvls = lambda b, e: list(starmap(pd.Interval,tuple(zip(b, e))))
    fmri_img = nimage.load_img(fmri_path)
    if not isinstance(events_path, pd.DataFrame):
        events = pd.read_csv(events_path, sep=sep)
    else:
        events = events_path
    t_r = [t_r if t_r is not None else
           fmri_img.header.get_zooms()[-1]][0]
    frame_times = np.arange(fmri_img.shape[-1]) * t_r
    frame_ends = pd.Series(frame_times).add(t_r).values
    frame_intervals = mkintrvls(pd.Series(frame_times).values,
                                frame_ends)
    trial_ends=(events.onset+abs(events.onset -
                                 events.offset)+events.isi).values
    trial_intervals = mkintrvls(events.onset.values, trial_ends)
    valid_trial_idx = [trial[0] for trial in enumerate(trial_intervals)
                       if trial[1].left<frame_intervals[-1].left]
    valid_trials = pd.Series(trial_intervals).loc[valid_trial_idx].values
#     trial_intervals = list(starmap(pd.Interval,tuple(zip(events.onset.values, trial_ends))))
    bold_by_trial_indx = [[frame[0] for frame in enumerate(frame_intervals)
                           if frame[1].left in trial] for trial in valid_trials]
    bold_by_trial = list(nimage.index_img(fmri_img, idx)
                         for idx in bold_by_trial_indx)
    valid_frame_intervals = [pd.Series(frame_intervals).loc[bold_idx].values
                             for bold_idx in bold_by_trial_indx]
    perfo_labels = events.iloc[valid_trial_idx].recognition_performance.fillna('Ctl')
    condition_labels = events.iloc[valid_trial_idx].trial_type
    stim_labels = events.iloc[valid_trial_idx].stim_file.fillna('Ctl').values
    categ_labels = events.iloc[valid_trial_idx].stim_category.fillna('Ctl').values
    return pd.DataFrame(tuple(zip(valid_trial_idx, bold_by_trial,
                                  bold_by_trial_indx, valid_trials,
                                  valid_frame_intervals, condition_labels,
                                  perfo_labels, stim_labels, categ_labels)),
                        columns=['trials', 'trial_niftis', 'fmri_frames',
                                 'trial_intervals', 'fmri_frame_intervals',
                                 'condition_labels', 'performance_labels',
                                 'stimuli_files', 'category_labels'])

In [ ]:
sub00.cleaned_fmri.get_fdata().shape

In [ ]:
testimg_data=sub00.cleaned_fmri.get_fdata().reshape(-1,77,60,117)
# testimg_data.shape

In [ ]:
from nilearn import datasets
from nilearn.input_data import NiftiMapsMasker
from sklearn.utils import Bunch
from get_difumo import get_difumo
atlases_dir = '../../nilearn_atlases/difumo_atlases/'
#/difumo_atlases/256/3mm/maps.nii.gz'

difumo_256 = get_difumo('../../nilearn_atlases/difumo_atlases/', 256, 3)

sub00.mapsmasker = NiftiMapsMasker(difumo_256.maps, mask_img=sub00.mask_img,
                                   resampling_target='mask',
                                   allow_overlap=True, **sub00.masker_defs)

In [ ]:
sub00.mapsmasker.fit(sub00.cleaned_fmri)
sub00.maps_signals = sub00.mapsmasker.transform_single_imgs(sub00.cleaned_fmri)

In [ ]:
sub00.maps_signals.shape#reshape(117,256)

In [ ]:
import nilearn 
sub00.vectors = trial_fmri(sub00.cleaned_fmri, sub00.events)
newshape=tuple(list(sub00.cleaned_fmri.shape)[:-1]+[117])
testimg = nilearn.image.resample_img(sub00.cleaned_fmri,
                                     target_affine=sub00.cleaned_fmri.affine,
                                     target_shape=newshape,
                                     interpolation='nearest',
                                     clip=True, force_resample=True)
# testmean = nilearn.image.concat_imgs([nilearn.image.mean_img(trial)
#                            for trial in sub00.vectors.trial_niftis])

In [ ]:
sub00.mapsmasker.fit(testmean)

In [ ]:
sub00.events.stim_category.replace({'K':'kitchen'}).fillna('Ctl').value_counts()

In [ ]:
# sub00.mapsmasker_mean = NiftiMapsMasker(difumo_256.maps, mask_img=sub00.mask_img,
#                                         resampling_target='mask',
#                                         allow_overlap=True, **sub00.masker_defs)
# sub00.mapsmasker_mean.fit(testmean)

In [ ]:
sub00.maps_mean_signals = sub00.mapsmasker.transform_single_imgs(testmean)

In [ ]:
# sub00.behav.stim_category.replace({'K':'kitchen'}).value_counts()

In [ ]:
#########################################################
# sub_svc = sklearn.svm.NuSVC() # 0.68 avg acc up to now
# sub_svc = sklearn.linear_model.SGDClassifier()
# 
# Cross-validation within 10 folds of training set
# predict
# class_weight='balanced'
cv_y_pred = cross_val_predict(sub_svc, X_train, y_train,
                              groups=y_train, cv=10)
# scores
cv_acc = cross_val_score(sub_svc, X_train, y_train,
                         groups=y_train, cv=10)
print(f'Cross-validation accuracy: {cv_acc}')

# evaluate overall model performance on training data
overall_acc = accuracy_score(y_pred = cv_y_pred, y_true = y_train)
overall_cr = classification_report(y_pred = cv_y_pred, y_true = y_train)
print(f'Accuracy\n{round(overall_acc, 3)}\n\nOverall Score\n{overall_cr}')

In [ ]:
sub00.events.recognition_performance.values.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import train_test_split

test_size=0.4
svc = LinearSVC(max_iter=100000)
signals = sub00.maps_mean_signals
# conditions = sub00.events.trial_type.values#.replace({'Enc':1,'Ctl':2}).values
conditions = sub00.events.recognition_performance.fillna('Ctl').values
# conditions = sub00.events.trial_type.replace({'Enc':'enc','Ctl':'ctl'}).astype('str')
groups = sub00.behav.recognition_performance.fillna('Ctl').values




In [ ]:
# Import it and define your fancy objects
from sklearn.feature_selection import RFE
svc = LinearSVC()
rfe = RFE(SVC(kernel='linear', C=1.), n_features_to_select=117, step=0.25)

# Create a new pipeline, composing the two classifiers `rfe` and `svc`

rfe_svc = Pipeline([('rfe', rfe), ('svc', svc)])

# Recompute the cross-validation score
cv_scores = cross_val_score(rfe_svc, signals, target, cv=cv, n_jobs=-1, verbose=1)
# But, be aware that this can take * A WHILE * ...

In [ ]:
# 1e-4<
5e-2

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
help(LinearSVC)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC
feature_selection = SelectPercentile(f_classif, percentile=10)
anova_svc = Pipeline([('anova', feature_selection), ('svc', LinearSVC(max_iter=10000000))])
# We can use our ``anova_svc`` object exactly as we were using our ``svc``
# object previously.
# As we want to investigate our model, we use sklearn `cross_validate` function
# with `return_estimator = True` instead of cross_val_score, to save the estimator

cv = LeaveOneGroupOut()
fitted_pipeline = cross_validate(anova_svc, signals, conditions,
                                 cv=cv, groups=groups, return_estimator=True)
print(
    "ANOVA+SVC test score: {:.3f}".format(fitted_pipeline["test_score"].mean()))


In [ ]:


from sklearn.model_selection import cross_val_score

# X_train, X_test, y_train, y_test = train_test_split(
#     signals, groups, # x & y
#     test_size = test_size, 
#     shuffle = True,
#     stratify = groups)
# svc.fit(X_train, y_train)

# svc.fit(signals, groups)
# Here `cv=5` stipulates a 5-fold cross-validation
cv_scores = cross_val_score(svc, signals,#sub00.maps_mean_signals,
                            groups,#sub00.vectors.condition_labels,
                            groups=conditions,
                            cv=5)

print("SVC accuracy: {:.3f}".format(cv_scores.mean()))

cv = LeaveOneGroupOut()
cv_scores = cross_val_score(svc, signals,
                            groups,
                            cv=cv,
                            scoring='roc_auc',
                            groups=conditions,
                            n_jobs=-1)

print("SVC accuracy (tuned parameters): {:.3f}".format(cv_scores.mean()))

# from sklearn.dummy import DummyClassifier
# null_cv_scores = cross_val_score(DummyClassifier(),
#                                  sub00.maps_mean_signals,
#                                  conditions,
#                                  cv=cv,
#                                  groups=groups)

# print("Dummy accuracy: {:.3f}".format(null_cv_scores.mean()))

# from sklearn.model_selection import permutation_test_score
# null_cv_scores = permutation_test_score(svc,
#                                         sub00.maps_mean_signals,
#                                         conditions,
#                                         cv=cv,
#                                         groups=groups)

# print("Permutation test score: {:.3f}".format(null_cv_scores.mean()))

In [ ]:
import SimpleITK as sitk
import os
import tensorflow as tf
import pandas as pd
import time

from matplotlib import pyplot as plt
from dltk.io.augmentation import *
from dltk.io.preprocessing import *

fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/'
events_dir = '/data/simexp/fnadeau/CIMAQ_AS_BIDS_4/'
%matplotlib inline

# Timer helper class for benchmarking reading methods
class Timer(object):
    """
    Timer class
    Wrap a will with a timing function
    """
    
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        self.t = time.time()
        
    def __exit__(self, *args, **kwargs):
        print("{} took {} seconds".format(
        self.name, time.time() - self.t))
# Set up parameters
batch_size = 1
iterations = sub00.testmean.shape[-1]

# Define the desired shapes and types of the training examples to pass to `read_fn`:
reader_params = {'n_examples': 1,
                 'example_size': list(sub00.testmean.shape),#[128, 224, 224],
                 'extract_examples': False}

reader_example_shapes = {'features': {'x': reader_params['example_size'] + [1,]},
                         'labels': {'y': []}}
 
reader_example_dtypes = {'features': {'x': tf.float32},
                         'labels': {'y': tf.int32}}


In [ ]:
# Method 1
def load_data(file_references, y, mode, params=None):
    
    data = {'features': [], 'labels': []}
    sitk_t1 = sitk.ReadImage(file_references)
    t1 = whitening(sitk.GetArrayFromImage(sitk_t1))
    data['features'].append(images)
    data['labels'].append(y.astype(np.int32))

    data['features'] = np.array(data['features'])
    data['labels'] = np.vstack(data['labels'])
    
    x = tf.placeholder(reader_example_dtypes['features']['x'], 
                       [*data['features'], 1])
    y = tf.placeholder(reader_example_dtypes['labels']['y'], 
                       [None, 1])            
    return x, y

# Load all data into memory
data = load_data(sub00.testmean, sub00.vectors.condition_labels,
                 tf.estimator.ModeKeys.TRAIN, params=reader_params)

In [ ]:


# x = tf.placeholder(reader_example_dtypes['features']['x'], 
#                    [*data['features'], 1])
# y = tf.placeholder(reader_example_dtypes['labels']['y'], 
#                    [None, 1])

dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.repeat(None)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(1)

# Check that features and labels dimensions match
assert features.shape[0] == labels.shape[0]

iterator = dataset.make_initializable_iterator()
nx = iterator.get_next()

with tf.train.MonitoredTrainingSession() as sess_dict:
    # Initialize iterator
    sess_dict.run(iterator.initializer,
                  feed_dict={x: data['features'],
                             y: data['labels']})
    
    with Timer('Feed dictionary'):
        # Timed feed dictionary example
        for i in range(iterations):
            # Get next features-labels pair
            dict_batch_feat, dict_batch_lbl = sess_dict.run(nx)



In [ ]:
# # Method 1
# def load_data(file_references, mode, params=None):
    
#     data = {'features': [], 'labels': []}
#     sitk_t1 = sitk.ReadImage(file_references)
#     t1 = whitening(sitk.GetArrayFromImage(sitk_t1))
#     data['features'].append(images)
#     data['labels'].append(y.astype(np.int32))

#     data['features'] = np.array(data['features'])
#     data['labels'] = np.vstack(data['labels'])
            
#     return data
    
    # We define a `read_fn` and iterate through the `file_references`, which
    # can contain information about the data to be read (e.g. a file path):
#     for meta_data in file_references:
        
        # Here, we parse the `sub_id` to construct a file path to read
        # an image from.
#         sub_id = meta_data[0]
#         data_path = '../../data/IXI_HH/1mm'
#         t1_fn = os.path.join(data_path, '{}/T1_1mm.nii.gz'.format(sub_id))
        
        # Read the .nii image containing a brain volume with SimpleITK and get 
        # the numpy array:
      

        # Normalise the image to zero mean/unit std dev:
#         t1 = whitening(t1)
        
        # Create a 4D Tensor with a dummy dimension for channels
#         t1 = t1[..., np.newaxis]

        # Labels: Here, we parse the class *sex* from the file_references 
        # \in [1,2] and shift them to \in [0,1] for training:
#         sex = np.int32(meta_data[1]) - 1
#         y = sex
#         y = np.int32(sub00.events.trial_type.values)
        
#         # If training should be done on image patches for improved mixing, 
#         # memory limitations or class balancing, call a patch extractor
#         if params['extract_examples']:
#             images = extract_random_example_array(
#                 t1,
#                 example_size=params['example_size'],
#                 n_examples=params['n_examples'])
            
#             # Loop the extracted image patches
#             for e in range(params['n_examples']):
#                 data['features'].append(images[e].astype(np.float32))
#                 data['labels'].append(y.astype(np.int32))
                     
#         # If desired (i.e. for evaluation, etc.), return the full images
#         else:
#             data['features'].append(images)
#             data['labels'].append(y.astype(np.int32))

#     data['features'] = np.array(data['features'])
#     data['labels'] = np.vstack(data['labels'])
            
#     return data

In [ ]:
tensortest=load_data(sub00.cleaned_fmri)

In [ ]:


######## Method 1 continued
# Visualise the `dict_batch_feat` using matplotlib.
input_tensor_shape = dict_batch_feat.shape
center_slices = [s//2 for s in input_tensor_shape]

# Visualise the `gen_batch_feat` using matplotlib.
f, axarr = plt.subplots(1, input_tensor_shape[0], figsize=(15,5));
f.suptitle(' '.join(['Visualisation of the `dict_batch_feat`',
                     f'input tensor with shape={input_tensor_shape}']))

for batch_id in range(input_tensor_shape[0]):
    # Extract a center slice image
    img_slice_ = np.squeeze(dict_batch_feat[batch_id, center_slices[1], :, :, :])
    img_slice_ = np.flip(img_slice_, axis=0)
    
    # Plot
    axarr[batch_id].imshow(img_slice_, cmap='gray');
    axarr[batch_id].axis('off')
    axarr[batch_id].set_title('batch_id={}'.format(batch_id))
    
f.subplots_adjust(wspace=0.05, hspace=0, top=0.8)
plt.show()

In [ ]:
def read_fn(file_references, mode, params=None):
    
    # We define a `read_fn` and iterate through the `file_references`, which
    # can contain information about the data to be read (e.g. a file path):
    for meta_data in file_references:
        
        # Here, we parse the `subject_id` to construct a file path to read
        # an image from.
        subject_id = meta_data[0]
        data_path = '../../data/IXI_HH/1mm'
        t1_fn = os.path.join(data_path, '{}/T1_1mm.nii.gz'.format(subject_id))
        
        # Read the .nii image containing a brain volume with SimpleITK and get 
        # the numpy array:
        sitk_t1 = sitk.ReadImage(t1_fn)
        t1 = sitk.GetArrayFromImage(sitk_t1)

        # Normalise the image to zero mean/unit std dev:
        t1 = whitening(t1)
        
        # Create a 4D Tensor with a dummy dimension for channels
        t1 = t1[..., np.newaxis]
        
        # If in PREDICT mode, yield the image (because there will be no label
        # present). Additionally, yield the sitk.Image pointer (including all
        # the header information) and some metadata (e.g. the subject id),
        # to facilitate post-processing (e.g. reslicing) and saving.
        # This can be useful when you want to use the same read function as 
        # python generator for deployment.
        if mode == tf.estimator.ModeKeys.PREDICT:
            yield {'features': {'x': t1}}

        # Labels: Here, we parse the class *sex* from the file_references 
        # \in [1,2] and shift them to \in [0,1] for training:
        sex = np.int32(meta_data[1]) - 1
        y = sex
        
        # If training should be done on image patches for improved mixing, 
        # memory limitations or class balancing, call a patch extractor
        if params['extract_examples']:
            images = extract_random_example_array(
                t1,
                example_size=params['example_size'],
                n_examples=params['n_examples'])
            
            # Loop the extracted image patches and yield
            for e in range(params['n_examples']):
                yield {'features': {'x': images[e].astype(np.float32)},
                       'labels': {'y': y.astype(np.int32)}}
                     
        # If desired (i.e. for evaluation, etc.), return the full images
        else:
            yield {'features': {'x': images},
                   'labels': {'y': y.astype(np.int32)}}

    return

In [ ]:
path00='/data/simexp/fnadeau/cimaq_2d/sub-3002498/ses-V10/func/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
# path00 = Path(path00)
# # dir(path00)
# # path00.split(path00.suffixes[0])[0]
# justname = str(path00).rstrip(''.join(path00.suffixes))

In [ ]:
def get_fmriprep_mask(fmri_path:Union[str,os.PathLike],
                      mask_ext:str='nii.gz', **kwargs):
    bids_patt = lambda p: f'(?<={p})[a-zA-Z0-9]*'
    prefixes = ['sub-','ses-','task-','space-']
    mask_sfx = '_'.join([pf+re.search(bids_patt(pf),
                                      os.path.basename(fmri_path)).group()
                         for pf in prefixes]+[f'desc-brain_mask.{mask_ext}'])
    return glob.glob(os.path.join(os.path.dirname(fmri_path), mask_sfx))[0]

def get_fmriprep_anat(fmri_path:Union[str,os.PathLike],
                      mdlt:str='T1w', ext:str='nii.gz',
                      **kwargs):
    space = '_space-'+re.search(f'(?<=_space-)[a-zA-Z0-9]*',
                      os.path.basename(fmri_path)).group()
    anat_suffix = f'*{space}_desc-preproc_{mdlt}.{ext}'
    return str(next(list(Path(fmri_path).parents)[2].rglob(anat_suffix)))

def get_events(fmri_path:Union[str,os.PathLike],
               events_dir:Union[str,os.PathLike])->str:
    sub_id, ses_id = Path(fmri_path).parts[-4:-2]
    globbed = glob.glob(os.path.join(events_dir,
                                  *Path(fmri_path).parts[-4:-2],
                                  '*events.tsv'))
    return [False if globbed == [] else globbed[0]][0]

        
def fetch_fmriprep_session(fmri_path:Union[str,os.PathLike],
                           events_dir:Union[str,os.PathLike],
                           strategy:str='Minimal',
                           anat_mod:str='T1w',
                           lc_kws:dict=None,
                           clean_kws:dict=None,
                           **kwargs):
    import load_confounds
    from inspect import getmembers
    from sklearn.utils import Bunch
    from pathlib import Path
    sub_id, ses_id = Path(fmri_path).parts[-4:-2]
    mask_path = get_fmriprep_mask(fmri_path)
    anat_path = get_fmriprep_anat(fmri_path)
    loader = dict(getmembers(load_confounds))[f'{strategy}']
    loader = [loader(**lc_kws) if lc_kws is not None
              else loader()][0]    
    event_path = get_events(fmri_path,events_dir)
    if event_path is False:
        return False
    else:
        return Bunch(**dict(sub_id=sub_id, ses_id=ses_id,
                            full_mask_path=mask_path,
                            fmri_path=fmri_path,
                            anat_path=anat_path,
                            event_path=event_path,
                            loader=loader,
                            confounds_strategy=strategy))

def load_fmriprep_session(fmri_path:Union[str,os.PathLike],
                          events_dir:Union[str,os.PathLike],
                          clean_kws:dict=None,
                          masker_kws:dict=None,
                          **kwargs):
    from nilearn import image as nimage
    from nilearn.masking import apply_mask, unmask
    from nilearn.signal import clean
    from sklearn.utils import Bunch
    from glob import glob
    from nilearn import image as nimage
    session = fetch_fmriprep_session(fmri_path,events_dir)
    fmri_img, mask_img, anat_img = \
        tuple(map(nimage.load_img,[session.fmri_path, session.full_mask_path,
                                   session.anat_path]))
    t_r = fmri_img.header.get_zooms()[-1]
    conf = session.loader.load(session.fmri_path)
    clean_defs = dict(standardize=False,
                      standardize_confounds=False,
                      high_pass=None, low_pass=None,
                      t_r=fmri_img.header.get_zooms()[-1],
                      ensure_finite=True)
    masker_defs = dict(mask_img=mask_img,
                       allow_overlap=True, t_r=t_r,
                       standardize_confounds=False,
                       resampling_target='mask')
    if masker_kws is not None:
        masker_defs.update(masker_kws)
    if clean_kws is not None:
        clean_defs.update(clean_kws)
    cleaned_fmri = unmask(clean(apply_mask(fmri_img, mask_img,
                                           smoothing_fwhm=8,
                                           dtype='f'),
                                confounds=conf,
                                **clean_defs),
                          mask_img)
    cleaned_fmri = nimage.new_img_like(fmri_img, cleaned_fmri.get_fdata(),
                               copy_header=True)
    events = pd.read_csv(session.event_path, sep='\t').iloc[1:,:]
    loaded = Bunch(**dict(fmri_img=fmri_img, cleaned_fmri=cleaned_fmri,
                          full_mask_img=mask_img, anat_img=anat_img,
                          clean_params=clean_defs, masker_params=masker_defs,
                          confounds=conf, events=events, t_r=t_r))
    loaded.update(session)
    return loaded

In [ ]:
# import numpy as np
# def rot90flip(data, rot_angle:int=None, scl, vol):
    
rot90flip=lambda a, s, v: np.rot90(a[:,:,s,v])
rot180flip=lambda a, s, v: np.rot90(np.rot90(a[:,:,s,v]))
rot270flip=lambda a, s, v: np.rot90(np.rot90(np.rot90(a[:,:,s,v])))
rot_dict = {90: rot90flip, 180: rot180flip, 270: rot270flip}
rot_dict[90]([0,1,2], 2, 3)
# np.rot90([[0,1,2]][:, :, 2, 33])
# import scipy
# dir(scipy)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# src = '../../flat_bold_png/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold_t306_z46.png'


# imageio.imread(src).shape
help(np.repeat)

# Image.open(src)._Image__transformer()
# dir(img._Image__transformer)
# skimage.show(input_image)

In [ ]:
from pathlib import Path
cimaq_stims = list(map(str, list(Path('../../WMStim').rglob('*.bmp'))))

In [ ]:
def preprocess_image(src:Union[str,os.PathLike]):
    # Open image
#     input_image = PIL.Image.fromarray(np.uint8(PIL.Image.open(src)))
#     input_image = PIL.Image.fromarray(PIL.Image.open(src))
    input_image = PIL.Image.open(src)
    # Preprocess image
    preprocess = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])])
    return preprocess(input_image).unsqueeze(0)
preprocess_image(cimaq_stims[0])

In [ ]:
help(imageio.imread)

In [ ]:
# len(list(filter(lambda x: x.endswith('.bmp'), cimaq_stims)))==len(cimaq_stims)
sorted(cimaq_stims)[6:]

In [ ]:
model = GoogLeNet.from_pretrained("googlenet")

def get_features(src, model):
#     input_tensor = preprocess_image(src)
#     model.eval()
    return model.extract_features(preprocess_image(src))
features = [get_features(src, model) for src in sorted(cimaq_stims)[6:]]
labels = list(map(os.path.basename, sorted(cimaq_stims)[6:]))
# list(get_features(cimaq_stims[x], model) for x in range(len(cimaq_stims)))
# help(model.eval)
# eval mode shape == torch.Size([1, 1024, 7, 7])


In [ ]:
apath = '../../flat_bold_png/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold_t306_z46.png'
from googlenet_pytorch import GoogLeNet
from typing import Union
# model = GoogLeNet.from_pretrained("ImageNet")

import numpy as np
import PIL
import skimage

import json
import os
import torch
from io import BytesIO
import torchvision.transforms as transforms
from PIL import Image
from keras.applications.vgg16 import VGG16
from googlenet_pytorch import GoogLeNet 


def load_inputs(src:Union[str,os.PathLike]):
    # Open image
#     input_image = PIL.Image.open(BytesIO(imageio.imread(src)))
#     model = VGG16()
    model = GoogLeNet.from_pretrained("googlenet")
    input_image = PIL.Image.fromarray(np.stack(np.repeat(np.uint8(PIL.Image.open(src)),3, axis=0)))
    # Preprocess image
    preprocess = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

    # Load class names
#     labels_map = json.load(open("labels_map.txt"))
#     labels_map = [labels_map[str(i)] for i in range(1000)]

    # Feature extraction
    # ... image preprocessing as in the classification example ...
    # inputs = torch.randn(1, 3, 224, 224)
#     print(inputs.shape) # torch.Size([1, 3, 224, 224])

    features = model.extract_features(input_batch)
    print(features.shape) # torch.Size([1, 1024, 7, 7])

    # Classify with GoogLeNet
    model.eval()

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to("cuda")
        model.to("cuda")

    with torch.no_grad():
        logits = model(input_batch)
        preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()
        print("-----")
        print(f'Predictions: {preds}')
        for idx in preds:
#             label = labels_map[idx]
            prob = torch.softmax(logits, dim=1)[0, idx].item()
#             print(f"{label:<75} ({prob * 100:.2f}%)")
    return input_tensor, input_batch, features, prob
test_tensors, test_batch, test_features, test_prob = load_inputs(cimaq_stims[0])

In [ ]:
test_prob

In [ ]:
import torch
from googlenet_pytorch import GoogLeNet
from numpy import asarray
model = GoogLeNet.from_pretrained('ImageNet')
apath = '../../flat_bold_png/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold_t306_z46.png'
from typing import Union
import json
import torchvision.transforms as transforms
from PIL import Image

from googlenet_pytorch import GoogLeNet 
# ... image preprocessing as in the classification example ...
# inputs = torch.randn(1, 3, 224, 224)
# inputs = sorted(map(str,list(Path(testsess).iterdir())))
# input_image = asarray(Image.open(apath))
preprocess = transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225])])
input_tensor = preprocess(Image.open(apath))
print(input_tensor.shape) # torch.Size([1, 3, 224, 224])

features = model.extract_features(input_tensor)
torch.Size([1, 1024, 7, 7])
print(features.shape)

In [ ]:
dir(Image.open(apath))

In [ ]:
# !pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
from googlenet_pytorch import GoogLeNet
model = GoogLeNet.from_pretrained("googlenet")

import json

import torch
import torchvision.transforms as transforms
from PIL import Image

from googlenet_pytorch import GoogLeNet 

In [ ]:
!pip install -U IProgress

In [ ]:
# 
import os
os.getcwd()

In [ ]:
len(os.listdir('sub-3163875_ses-V03_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold'))

In [ ]:
flatdir = '../../flat_bold_png/'
from itertools import starmap
from pathlib import Path
testsess = sorted(Path(flatdir).iterdir())[0]

In [ ]:
str(list(Path(testsess).iterdir())[0])